In [ ]:
# Imports
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
from bs4 import BeautifulSoup 
import mplcyberpunk
#from tabulate import tabulate

In [ ]:
#extract the data from the website
url = requests.get("https://www.fundsexplorer.com.br/ranking")

In [ ]:
fii_bs4 = BeautifulSoup(url.text, "html.parser")

In [ ]:
#find the table with a tag
fii_table = fii_bs4.findAll(attrs = {'id' : 'scroll-wrapper'})
table = fii_table[0].findAll('table')

In [ ]:
fii_df = pd.read_html(str(fii_table[0]))[0]

In [ ]:
#initial dataframe
fii_df

In [ ]:
fii_df.columns.values

In [ ]:
#rename the columns
fii_df.columns = ['ticker', 'Setor', 'price', 'liquidez',
       'Dividendo', 'DividendYield', 'DY (3M)Acumulado',
       'DY (6M)Acumulado', 'dy', 'DY (3M)Média',
       'DY (6M)Média', 'Dividenmediio', 'DY Ano', 'Variação Preço',
       'Rentab.Período', 'Rentab.Acumulada', 'PatrimônioLíq.', 'VPA',
       'pvpa', 'DYPatrimonial', 'VariaçãoPatrimonial',
       'Rentab. Patr.no Período', 'Rentab. Patr.Acumulada',
       'VacânciaFísica', 'VacânciaFinanceira', 'QuantidadeAtivos']

In [ ]:
fii_df.columns.values

In [ ]:
#save a backup od the data
fii_back = fii_df.copy()

In [ ]:
#select the columns needed for the test
test = fii_df[['ticker', 'price', 'dy', 'pvpa','liquidez']]

In [ ]:
print("--> DATA:")
display(test.isnull().sum())

In [ ]:
#excludes the "nan" data
test = test[test['price'].notna()]
test = test[test['dy'].notna()]

In [ ]:
print("--> DATA:")
display(test.isnull().sum())

In [ ]:
#classified the dataframe by dividend yeld from largest to smallest 
test = test.sort_values(['dy'], ascending=False)

In [ ]:
test

In [ ]:
# excludes the "R$" unit from the data
unit = 'R$'
test['price'] = [sub.replace(unit, "").strip() for sub in test['price']]

In [ ]:
test

In [ ]:
#view the types of the columns
test.dtypes

In [ ]:
#change the separator digit of "," for "."
test['price'] = test['price'].str.replace(',', '.')
test['price'] = pd.to_numeric(test['price'], errors='coerce')
test

In [ ]:
#excludes the "nan" data
test = test[test['price'].notna()]

In [ ]:
#calculate the daily liquidity
test['lid2'] = test['price'] * test['liquidez']
test

In [ ]:
##classified the dataframe by daily liquidity from largest to smallest 
test = test.sort_values(['lid2'], ascending=False)

In [ ]:
#reset the index
test.reset_index(inplace=True, drop=True)

In [ ]:
#excludes the reits (fii) with daily liquidity less than R$ 200,000.00
indexNames = test[ (test['lid2'] <= 200000)].index
test.drop(indexNames , inplace=True)
test

In [ ]:
#save the number of lines
ts = test.shape
ts = ts[0]

In [ ]:
#classified the dataframe by dividend yeld from largest to smallest
test = test.sort_values(['dy'], ascending=False)
test.reset_index(inplace=True, drop=True)

In [ ]:
b1 = [*range(0, ts, 1)]

In [ ]:
#creates a ranking from highest to lowest dividend
test['b1'] = pd.Series(b1)

In [ ]:
test

In [ ]:
#classified the dataframe by pvpa from largest to smallest
test = test.sort_values(['pvpa'], ascending=True)

In [ ]:
test.reset_index(inplace=True, drop=True)

In [ ]:
#creates a ranking from highest to lowest pvpa
b2 = [*range(0, ts, 1)]
test['b2'] = pd.Series(b2)

In [ ]:
test

In [ ]:
#sum the dividend yeld rank with the pvpa rank
test = test.eval('Rank = b1 + b2')

In [ ]:
#classified the dataframe by final rank from smallest to largest
test = test.sort_values(['Rank'], ascending=True)

In [ ]:
test.reset_index(inplace=True, drop=True)

In [ ]:
#select the top 10 reits (fii) to invest
indexNames = test[(test.index > 11)].index
test.drop(indexNames , inplace=True)
test

In [ ]:
indexNames = test[(test['ticker']=='PATL11')].index
test.drop(indexNames , inplace=True)
indexNames = test[(test['ticker']=='HSLG11')].index
test.drop(indexNames , inplace=True)
test.reset_index(inplace=True, drop=True)
test

In [ ]:
y = [10,10,10,10,10,10,10,10,10,10]
ml = test['ticker']
me = [0.2,0,0,0,0,0,0,0,0,0]
colors = plt.cm.rainbow(np.linspace(0, 1, len(y)))
plt.style.use("cyberpunk")
plt.pie(y, labels = ml, shadow=True, autopct='%1.1f%%', explode=me, colors=colors)
plt.title('REITS RANK',fontname="Arial",fontweight="bold", size=20)
#plt.legend()

In [ ]:
tts = test['ticker']+".SA"
print(tts)

In [ ]:
x = np.random.rand(10,1)
#x = x.T
x = pd.DataFrame(x, columns=[f'name{0}'])


for i in range (0,10):
    y = np.random.rand(10,1)
    #y = y.T
    y = pd.DataFrame(y, columns=[f'name{i+1}'])
    x = pd.concat([x,y], axis=1, join='inner', ignore_index=False, copy=False)

x


In [ ]:
ifix = "XFIX11.SA"
data = yf.download(ifix, period='1y')
dd = pd.DataFrame(data)
dd.drop(['Open','High','Low','Close','Volume'], axis=1, inplace=True)
dd.rename(columns={'Adj Close': ifix}, inplace=True)

for x in range (10):
    data = yf.download(tts[x], period='1y')
    ddsad = pd.DataFrame(data)
    ddsad.drop(['Open','High','Low','Close','Volume'], axis=1, inplace=True)
    ddsad.rename(columns={'Adj Close': tts[x]}, inplace=True)
    dd = pd.concat([dd, ddsad], axis=1, join='outer', ignore_index=False, copy=False)
dd
#dd

In [ ]:
dd = dd.resample("M").last()
dd = dd.pct_change()
dd = dd.dropna(how='all')
dd.drop(['BCFF11.SA'], axis=1, inplace=True)
dd = dd*100
dd

In [ ]:
#sns.heatmap(dd,annot=True)
#dd.plot_monthly_heatmap()

plt.style.use("cyberpunk")
plt.plot(dd.index,dd)
plt.grid(which='minor', alpha=0.2)
plt.grid(which='major', alpha=0.5)
plt.legend(dd.columns,loc='best', shadow=True, frameon=True)
#ax.set_label(y)
#ax.set_xticks(ind,labels=x)
#ax.bar_label(f, fmt='%.1f',padding=4, fontweight="bold")

plt.xlabel('MONTH', fontweight="bold")
plt.ylabel('VARIAÇÃO', fontweight="bold", visible=False)
plt.title('ANNUAL RETURN (%)', fontweight="bold")
plt.rcParams['figure.figsize'] = [10, 10]


In [ ]:
sns.heatmap(dd, annot=True)

In [ ]:
sumdd = dd.sum()
sumdd = pd.DataFrame(sumdd)
x = sumdd.index
y = sumdd[0]
colors = plt.cm.rainbow(np.linspace(0, 1, len(y)))

fig,ax = plt.subplots()
f = ax.bar(x,y,color=colors)
plt.xticks(x,rotation=90)
ax.set_label(y)
ax.bar_label(f,fmt='%.1f',padding=4, fontweight="bold")
plt.xlabel('REITS', fontweight="bold")
plt.ylabel('VARIATION', fontweight="bold", visible=False)
plt.title('ACUMULATED RETURN (%)', fontweight="bold")